In [1]:
import pickle
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set()

from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LogisticRegressionCV
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn import preprocessing
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import PolynomialFeatures

from imblearn.over_sampling import SMOTE

In [2]:
#load Hugo's RF model
rf_pkl = open('../hugo/Random_Forest_Model.pkl', 'rb')
rf_pipeline = pickle.load(rf_pkl);
#extract the RF model from the Pipeline object
rf_model = rf_pipeline.steps[1][1];

In [3]:
rf_trees = rf_model.estimators_;
#count top feature for Random Forest model
rf_top_features = {};
for tree in rf_trees:
    for i in range(7):
        top_feat = tree.tree_.feature[i];
        try:
            rf_top_features[top_feat] += 1;
        except:
            rf_top_features[top_feat] = 1;
        
# print("Random Forest Top Features", rf_top_features);
print(len(rf_top_features.keys()), "top features in random forest")

180 top features in random forest


In [4]:
top_feats_sorted = sorted(rf_top_features.items(), key= lambda x: x[1], reverse=True);
print(top_feats_sorted)

[(1, 28), (88, 26), (92, 23), (120, 21), (90, 16), (5, 16), (175, 14), (198, 12), (109, 10), (273, 10), (234, 10), (24, 10), (25, 9), (271, 9), (194, 9), (844, 9), (112, 9), (215, 9), (254, 9), (45, 8), (49, 8), (274, 8), (190, 8), (0, 8), (282, 8), (824, 8), (174, 7), (64, 7), (68, 7), (235, 7), (105, 7), (107, 7), (153, 7), (269, 6), (761, 6), (70, 6), (115, 6), (111, 6), (91, 6), (256, 6), (108, 5), (762, 5), (69, 5), (276, 5), (3, 5), (833, 5), (71, 5), (131, 5), (689, 5), (267, 5), (119, 5), (173, 5), (46, 4), (110, 4), (214, 4), (277, 4), (690, 4), (134, 4), (734, 4), (44, 4), (238, 4), (228, 4), (113, 4), (230, 4), (195, 4), (157, 3), (881, 3), (94, 3), (237, 3), (77, 3), (162, 3), (47, 3), (149, 3), (76, 3), (683, 3), (956, 3), (154, 3), (275, 3), (62, 3), (863, 3), (114, 3), (48, 3), (4, 3), (73, 3), (2, 3), (193, 2), (272, 2), (151, 2), (348, 2), (21, 2), (196, 2), (27, 2), (155, 2), (89, 2), (692, 2), (898, 2), (852, 2), (281, 2), (156, 2), (344, 2), (150, 2), (236, 2), (784

In [5]:
#create dataframe that Hugo trained the model on again
df = pd.read_csv("../hugo/cleaned_2014", low_memory = False);
data_train, data_test = train_test_split(df, test_size=.2, stratify=df.paid, random_state=1)
def split_data(df, cols):
    x = df.drop(cols, axis = 1)
    y = df.paid
    return x, y

cols_to_drop_training = ['loan_status', 'paid', 'amnt', 'total_pymnt', 'term']
x_train_initial, y_train_initial = split_data(data_train, cols_to_drop_training)

In [6]:
poly = PolynomialFeatures(degree=2, interaction_only=False, include_bias=False);
poly.fit(x_train_initial);
all_training_features = poly.get_feature_names(input_features= x_train_initial.columns);

In [7]:
print("Training Features: {0}".format(len(all_training_features)))

for i in range(10):
    feat_num = top_feats_sorted[i][0];
    print("Feature {0}: {1}".format(feat_num, all_training_features[feat_num]))


Training Features: 989
Feature 1: int_rate
Feature 88: int_rate dti
Feature 92: int_rate emp_length_2 years
Feature 120: int_rate purpose_major_purchase
Feature 90: int_rate emp_length_1 year
Feature 5: emp_length_1 year
Feature 175: dti emp_length_4 years
Feature 198: dti purpose_debt_consolidation
Feature 109: int_rate grade_B
Feature 273: emp_length_1 year purpose_car
